# 1 Non-Performing Loan Flag Prediction from Credit Card Information

Each year, non-performing loans cost the economy millions of baht. Predicting whether a
customer will default on their credit card loans is a non-trivial task, which if solved, can be
highly beneficial to both KBank and KBank’s customers. If default could be predicted in
advance, loans could be restructured, this would save customers from bankruptcy, and prevent
KBank from losing revenue.

The task is to use the data given to predict if customers defaulted on their credit card
loans in a certain period of time.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Beautiful format for float type
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## CSV to DataFrame

In [4]:
card_details = pd.read_csv('tj_01_creditcard_card.csv')
customer_details = pd.read_csv('tj_01_creditcard_customer.csv')
transactions = pd.read_csv('tj_01_creditcard_transaction.csv')

mcc_codes = pd.read_csv('mcc_codes.csv')

data_train = pd.read_csv('tj_01_training.csv', header = None, names = ['card_no', 'npl_flag'])
data_test = pd.read_csv('tj_01_test.csv', header = None, names = ['card_no'])

In [ ]:
data_train[data_train.groupby('card_no').count().reset_index()['npl_flag'] > 1]

In [5]:
card_details = card_details.dropna()
card_details.isnull().values.any()

False

In [6]:
customer_details = customer_details.fillna(0)
customer_details.isnull().values.any()

False

In [7]:
transactions = transactions.fillna(0)
transactions.isnull().values.any()

False

In [8]:
mcc_codes = mcc_codes.fillna("0")
mcc_codes.isnull().values.any()

False

## Merge Related DataFrame

In [9]:
card_details = card_details.merge(customer_details, left_on = 'cst_id', right_on = 'cst_id', how = "left")
card_details.sample()

,card_no,bill_cyc,pos_dt_x,cst_id,open_dt,exp_dt,cr_lmt_amt,prev_cr_lmt_amt,incm_amt,age,main_zip_cd,cr_line_amt,pos_dt_y
9152,1234000000025202,20,2017-02-23 00:00:00,1000011886,2015-06-03 00:00:00,620,24000.000,0.000,17000,27.000,12130,24000.000,2017-02-23 00:00:00


In [10]:
transactions = transactions.merge(mcc_codes, how = "left", left_on = "mer_cat_code", right_on = "mcc")
transactions.sample(3)

,card_no,txn_date,txn_hour,txn_amount,mer_cat_code,mer_id,mcc,edited_description,combined_description,usda_description,irs_description,irs_reportable
200758,1234000000023380,2016-03-31 00:00:00,8,350.000,4111,0,4111.000,Local/Suburban Commuter Passenger Transportati...,Local/Suburban Commuter Passenger Transportati...,Local/Suburban Commuter Passenger Transportati...,"Commuter Transport, Ferries",Yes
526275,1234000000011427,2016-08-05 00:00:00,12,1050.000,5812,24875,5812.000,Eating places and Restaurants,Eating places and Restaurants,Eating places and Restaurants,"Eating Places, Restaurants",No1.6041-3(c)
174108,1234000000015812,2016-08-06 00:00:00,14,200.000,5411,0,5411.000,"Grocery Stores, Supermarkets","Grocery Stores, Supermarkets",Grocery Stores,"Grocery Stores, Supermarkets",No1.6041-3(c)


## Cleanup Cards

In [ ]:
card_details.sample()

In [11]:
def cleanup_card_columns(df):
    drop_columns = ['pos_dt_x'
                    , 'cst_id'
                    , 'open_dt'
                    , 'exp_dt'
                    , 'pos_dt_y']
    
    return df.drop(drop_columns, axis = 1)


def make_readable_columns(df):
    df.columns = ['card_no'
                  , 'bill_cycle'
                  , 'credit_limit'
                  , 'prev_credit_limit'
                  , 'income'
                  , 'age'
                  , 'zip'
                  , 'credit_available']
    
    return df


def cleanup_cards(df):
    df = cleanup_card_columns(df)
    df = make_readable_columns(df)
    
    return df


clean_card_details = cleanup_cards(card_details)
clean_card_details.sample()

,card_no,bill_cycle,credit_limit,prev_credit_limit,income,age,zip,credit_available
8189,1234000000022813,17,150000.000,0.000,75766,28.000,10250,150000.000


## Cleanup Transactions

In [ ]:
transactions.sample()

In [12]:
def cleanup_transaction_columns(df):
    drop_columns = ['mer_cat_code'
                    , 'mer_id'
                    , 'mcc'
                    , 'txn_hour'
                    , 'combined_description'
                    , 'usda_description'
                    , 'irs_description'
                    , 'irs_reportable']
    
    return df.drop(drop_columns, axis = 1)


def make_readable_columns(df):
    df.columns = ['card_no'
                  , 'date'
                  , 'amount'
                  , 'merchant']
    
    return df


def simplify_date(df):
    df['date'] = pd.to_datetime(df['date']).map(lambda x: x.strftime('%Y-%m'))
    
    return df


def cleanup_transactions(df):
    df = cleanup_transaction_columns(df)
    df = make_readable_columns(df)
    df = simplify_date(df)
    
    return df


clean_transactions = cleanup_transactions(transactions)
clean_transactions.sample()

,card_no,date,amount,merchant
28068,1234000000014886,2016-08,350.000,Eating places and Restaurants


## Visualizing Data
### Card Detail

In [ ]:
sns.barplot(data = clean_card_details, x = 'age', y = 'credit_limit')

In [ ]:
sns.barplot(data = clean_card_details, x = 'age', y = 'income')

### Transaction

In [ ]:
# sns.distplot(data = clean_transactions, x = 'date', y = 'amount')

## Feature Engineering

### Use amount per month

In [13]:
transaction_amount_per_month = clean_transactions.groupby(['card_no', 'date']).sum().reset_index()
transaction_amount_per_month.head(6)

,card_no,date,amount
0,1234000000000004,2016-01,9500.000
1,1234000000000004,2016-02,3200.000
2,1234000000000004,2016-03,5200.000
3,1234000000000004,2016-04,4500.000
4,1234000000000004,2016-05,4400.000
5,1234000000000004,2016-06,2700.000


### Max amount

In [14]:
transaction_max = transaction_amount_per_month.groupby(['card_no']).max().reset_index()
transaction_max = transaction_max.fillna(0).drop('date', axis = 1)
transaction_max.columns = ['card_no', 'max_amount']
transaction_max.head(6)

,card_no,max_amount
0,1234000000000004,9500.000
1,1234000000000005,750.000
2,1234000000000006,77200.000
3,1234000000000010,7100.000
4,1234000000000012,13250.000
5,1234000000000014,22350.000


## Min amount

In [15]:
transaction_min = transaction_amount_per_month.groupby(['card_no']).min().reset_index()
transaction_min = transaction_min.fillna(0).drop('date', axis = 1)
transaction_min.columns = ['card_no', 'min_amount']
transaction_min.head(6)

,card_no,min_amount
0,1234000000000004,2050.000
1,1234000000000005,100.000
2,1234000000000006,1750.000
3,1234000000000010,1700.000
4,1234000000000012,500.000
5,1234000000000014,450.000


### Mean

In [16]:
transaction_mean = transaction_amount_per_month.groupby(['card_no']).mean().reset_index().fillna(0)
transaction_mean.columns = ['card_no', 'mean_amount']
transaction_mean.head(6)

,card_no,mean_amount
0,1234000000000004,4868.750
1,1234000000000005,312.500
2,1234000000000006,18800.000
3,1234000000000010,3756.250
4,1234000000000012,7406.250
5,1234000000000014,6587.500


### Diff Credit Limit  

In [17]:
clean_card_details_with_diff = pd.DataFrame(columns = ['card_no', 'diff_credit_limit'])
clean_card_details_with_diff.card_no = clean_card_details.card_no
clean_prev_credit_limit = clean_card_details.prev_credit_limit.apply(lambda x: 1 if x == 0 else x)
clean_card_details_with_diff.diff_credit_limit = (clean_card_details.credit_limit \
                                                / clean_prev_credit_limit)
clean_card_details_with_diff.sample(6)

,card_no,diff_credit_limit
532,1234000000023808,0.708
6883,1234000000009806,60000.000
253,1234000000002576,0.833
2930,1234000000020697,0.314
10498,1234000000005555,2.000
6900,1234000000002770,1.555


### Pivot Merchant in transactions

In [39]:
clean_transactions.sample(4)

,card_no,date,amount,merchant
475308,1234000000011958,2016-05,450.000,Misc. Food Stores – Convenience Stores and Spe...
174375,1234000000016887,2016-07,200.000,"Insurance Sales, Underwriting, and Premiums"
254502,1234000000025188,2016-02,350.000,Department Stores
271197,1234000000007271,2016-07,350.000,"Insurance Sales, Underwriting, and Premiums"


In [18]:
transaction_merchants = clean_transactions.drop(['date'], axis = 1)
# transaction_merchants = transaction_merchants.groupby(['card_no', 'merchant']).sum()

transaction_merchants = pd.pivot_table(transaction_merchants
                                       , values='amount'
                                       , index=['card_no']
                                       , columns=['merchant']
                                       , aggfunc=np.sum).fillna(0).reset_index()
transaction_merchants.sample(3)

merchant,card_no,AEORFLOT,AEROMEXICO,AIR CANADA,AIR CHINA,AIR FRANCE,AIR NEW ZEALAND,ALAMO RENT-A-CAR,ALASKA AIRLINES,ALL NIPPON AIRWAYS,...,Video Tape Rental Stores,Vocational Schools and Trade Schools,WESTIN HOTELS,"Watch, Clock, Jewelry, and Silverware Stores","Watch, Clock, and Jewelry Repair",Welding Repair,Wholesale Clubs,Wig and Toupee Stores,Women’s Accessory and Specialty Shops,Women’s Ready-to-Wear Stores
3229,1234000000007761,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,4750.000,0.000,0.000,0.000,0.000,0.000,0.000
7748,1234000000018326,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1150.000,0.000
11232,1234000000026684,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,11100.000,0.000,0.000,0.000,0.000,0.000,0.000


### Merge to cards

In [19]:
merge_clean_card_details = clean_card_details.join(transaction_mean.set_index('card_no'), on = 'card_no', how = 'left')
merge_clean_card_details = merge_clean_card_details.join(transaction_min.set_index('card_no'), on = 'card_no', how = 'left')
merge_clean_card_details = merge_clean_card_details.join(transaction_max.set_index('card_no'), on = 'card_no', how = 'left')
merge_clean_card_details = merge_clean_card_details.join(clean_card_details_with_diff.set_index('card_no'), on = 'card_no', how = 'left')
merge_clean_card_details = merge_clean_card_details.join(transaction_merchants.set_index('card_no'), on = 'card_no', how = 'left')
merge_clean_card_details.sample(6)

,card_no,bill_cycle,credit_limit,prev_credit_limit,income,age,zip,credit_available,mean_amount,min_amount,...,Video Tape Rental Stores,Vocational Schools and Trade Schools,WESTIN HOTELS,"Watch, Clock, Jewelry, and Silverware Stores","Watch, Clock, and Jewelry Repair",Welding Repair,Wholesale Clubs,Wig and Toupee Stores,Women’s Accessory and Specialty Shops,Women’s Ready-to-Wear Stores
3890,1234000000023736,20,120000.000,0.000,101695,33.000,84320,120000.000,1850.000,600.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4304,1234000000001670,20,80000.000,0.000,26000,54.000,10140,80000.000,3925.000,400.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
790,1234000000002611,20,28000.000,33600.000,16500,35.000,10600,28000.000,5112.500,750.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,250.000
9645,1234000000020466,17,147000.000,50000.000,42000,31.000,66000,147000.000,7750.000,650.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
214,1234000000000108,20,80000.000,90000.000,15000,62.000,10900,150000.000,3912.500,950.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6292,1234000000000272,20,40000.000,0.000,20000,35.000,41320,40000.000,3587.500,850.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### Diff income and  mean

In [20]:
diff_card_details = merge_clean_card_details.copy()
# clean_card_details.income.astypeype('int64')
# clean_card_details.mean.astype('int64')
diff_card_details['diff_income_amount'] = diff_card_details.income \
                                                - diff_card_details.mean_amount
diff_card_details.sample(6)

,card_no,bill_cycle,credit_limit,prev_credit_limit,income,age,zip,credit_available,mean_amount,min_amount,...,Vocational Schools and Trade Schools,WESTIN HOTELS,"Watch, Clock, Jewelry, and Silverware Stores","Watch, Clock, and Jewelry Repair",Welding Repair,Wholesale Clubs,Wig and Toupee Stores,Women’s Accessory and Specialty Shops,Women’s Ready-to-Wear Stores,diff_income_amount
10253,1234000000002324,20,68000.000,0.000,34058,36.000,10400,68000.000,3631.250,1150.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2000.000,0.000,30426.750
2345,1234000000023578,17,130000.000,115000.000,57880,27.000,11120,130000.000,683.333,300.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,57196.667
2086,1234000000018229,5,400000.000,200000.000,261036,58.000,12000,400000.000,5656.250,850.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,255379.750
1311,1234000000017350,17,200000.000,0.000,109629,39.000,12120,200000.000,7033.333,200.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,102595.667
2874,1234000000015302,25,348000.000,0.000,174353,59.000,10210,348000.000,5100.000,2600.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,169253.000
10690,1234000000027068,17,40000.000,0.000,21970,66.000,45000,40000.000,400.000,400.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,21570.000


In [42]:
final_card_details = diff_card_details.copy()
final_card_details['income_mean_ratio'] = final_card_details.income / final_card_details.mean_amount
final_card_details['income_max_ratio'] = final_card_details.income / final_card_details.max_amount
final_card_details['income_min_ratio'] = final_card_details.income / final_card_details.min_amount

final_card_details = final_card_details.drop(['bill_cycle'
                                              , 'credit_limit'
                                              , 'prev_credit_limit'
                                              , 'credit_available']
                                             , axis = 1)
final_card_details = final_card_details.fillna(0)
final_card_details = final_card_details.drop(['mean_amount', 'min_amount', 'max_amount', 'zip'], axis = 1)
final_card_details.describe()

,card_no,income,age,diff_credit_limit,AEORFLOT,AEROMEXICO,AIR CANADA,AIR CHINA,AIR FRANCE,AIR NEW ZEALAND,...,"Watch, Clock, and Jewelry Repair",Welding Repair,Wholesale Clubs,Wig and Toupee Stores,Women’s Accessory and Specialty Shops,Women’s Ready-to-Wear Stores,diff_income_amount,income_mean_ratio,income_max_ratio,income_min_ratio
count,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,...,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000,11875.000
mean,1234000000014100.500,70396.859,41.080,60004.235,0.417,0.008,0.051,7.676,2.072,0.008,...,3.853,1.423,2646.939,3.739,492.863,358.295,56339.600,24.408,17.518,130.746
std,8130.274,112634.628,10.433,94299.221,34.692,0.918,4.104,780.567,153.717,0.918,...,122.281,104.409,31188.300,278.282,4315.542,2200.293,119708.103,110.595,103.893,375.648
min,1234000000000004.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,-3397658.500,0.000,0.000,0.000
25%,1234000000007142.500,24824.500,33.000,0.861,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,16863.375,3.259,1.291,14.439
50%,1234000000014085.000,42347.000,39.000,30000.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,32762.750,6.190,2.643,33.810
75%,1234000000021072.000,77373.500,48.000,81000.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,63885.000,12.568,5.995,99.064
max,1234000000028171.000,3859715.000,117.000,1000000.000,3700.000,100.000,400.000,84900.000,12300.000,100.000,...,7750.000,10400.000,1217400.000,28450.000,245000.000,77350.000,3839065.000,3191.140,3191.140,10774.700


## Splitting up the training Data

In [43]:
data_train_with_details = data_train.join(final_card_details.set_index('card_no'), on = 'card_no', how = 'left')
data_train_with_details.sample()

,card_no,npl_flag,income,age,diff_credit_limit,AEORFLOT,AEROMEXICO,AIR CANADA,AIR CHINA,AIR FRANCE,...,"Watch, Clock, and Jewelry Repair",Welding Repair,Wholesale Clubs,Wig and Toupee Stores,Women’s Accessory and Specialty Shops,Women’s Ready-to-Wear Stores,diff_income_amount,income_mean_ratio,income_max_ratio,income_min_ratio
4629,1234000000017679,0,35144,33.000,61000.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,26575.250,4.101,0.666,78.098


In [32]:
from sklearn.model_selection import train_test_split

X_all = data_train_with_details.drop(['card_no', 'npl_flag'], axis=1)
y_all = data_train_with_details['npl_flag']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

In [33]:
X_train.shape

(7600, 354)

In [ ]:
X_train.describe()

## Fitting and Tuning an Classifier

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
random_forest = RandomForestClassifier()

# Choose some parameter combinations to try
parameters = {'n_estimators': [500], 
#               'max_features': ['log2', 'sqrt', 'auto'], 
#               'criterion': ['entropy', 'gini'],
#               'max_depth': [2, 3, 5, 10], 
#               'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [5]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(random_forest, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
random_forest = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [38]:
predictions = random_forest.predict(X_test)
print(accuracy_score(y_test, predictions))

0.785789473684


## Validate with KFold

In [39]:
from sklearn.cross_validation import KFold

clf = random_forest

def run_kfold(clf):
    kf = KFold(891, n_folds=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_kfold(clf)

Fold 1 accuracy: 0.7
Fold 2 accuracy: 0.797752808988764
Fold 3 accuracy: 0.8651685393258427
Fold 4 accuracy: 0.7640449438202247
Fold 5 accuracy: 0.7528089887640449
Fold 6 accuracy: 0.7528089887640449
Fold 7 accuracy: 0.6966292134831461
Fold 8 accuracy: 0.8426966292134831
Fold 9 accuracy: 0.7640449438202247
Fold 10 accuracy: 0.8089887640449438
Mean Accuracy: 0.7744943820224719


## Model Version
### Save model to file

In [28]:
from sklearn.externals import joblib

version_name = input('version name: ')

joblib.dump(clf, 'model_{}.pkl'.format(version_name)) 

version name: 774494


['model_774494.pkl']

### Load model from file

In [40]:
from sklearn.externals import joblib

version_name = input('version name: ')

clf = joblib.load('model_{}.pkl'.format(version_name)) 

version name: 779013


## Predict the Actual Test Data

In [45]:
test = data_test.join(final_card_details.set_index('card_no'), on = 'card_no', how = 'left')
predictions = clf.predict(test.drop('card_no', axis = 1))

output = pd.DataFrame({'npl_flag': predictions })
output.to_csv('1.txt', index = False, header = None)
output.head(10)

,npl_flag
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
